In [29]:
import os, re
import pandas as pd
import numpy as np

# #显示所有列
# pd.set_option('display.max_columns', None)

# #显示所有行
# pd.set_option('display.max_rows', None)

pd.set_option('max_colwidth',100) 

In [30]:
folders = [
    # r'D:\code\forecast_model\notebook\20250811_base',
    # r'D:\code\forecast_model\notebook\20250811_small_large',
    # r'D:\code\forecast_model\notebook\20250811_extraLarge',
    # r"D:\code\forecast_model\notebook\20250815_base_large",
    # r'D:\code\forecast_model\notebook\20250817_base',
    # r'D:\code\forecast_model\notebook\20250820_base',
    # r'D:\code\forecast_model\notebook\20250820_bigger',
    # r'D:\code\forecast_model\notebook\20250821_mlp_base2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline3',
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline2",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline3",
    # r'D:\code\forecast_model\notebook\20250825_tcn_baseline',
    # r"D:\code\forecast_model\notebook\20250826_tcn_filt_days",
    # r'D:\code\forecast_model\notebook\20250828_tcn_data',
    # r'D:\code\forecast_model\notebook\20250829_tcn_data',
    # r"D:\code\forecast_model\notebook\20250829_tcn_data2",
    # r'D:\code\forecast_model\notebook\20250830_tcn_data',
    # r'D:\code\forecast_model\notebook\20250831_tcn_only30',
    # r'D:\code\forecast_model\notebook\20250831_nofilter_top5',
    # r'D:\code\forecast_model\notebook\20250831_tcn_we',
    r'D:\code\forecast_model\notebook\20250831_tcn_we2',
    ]

In [31]:
data = pd.DataFrame()
for folder in folders:
    for i in os.listdir(folder):
        if 'muilt_model' == i:continue
        # 20250811_base_P100_DeepLOB_v2_input_indepent	0.558035	2.77h
        # 20250811_base_P100_DeepLOB_v2_overfit
        if 'input_indepent' in i or 'overfit' in i: continue

        if i.endswith('.csv'): continue
        if i.endswith('.xlsx'): continue

        code = i.split('_')[3]
        fold = i.split('_')[4]

        train_folder = os.path.join(folder, i)
        if not os.path.isdir(train_folder): continue

        result_file = os.path.join(train_folder, 'result.csv')
        if not os.path.exists(result_file): continue

        # 修复替换
        _data = open(result_file, 'r').readlines()
        _data[1] = _data[1].replace('(30, 2, 1)', '30@2@1')
        with open(result_file, 'w') as f:
            f.writelines(_data)

        _data = pd.read_csv(result_file)
        # _data['describe.1'] = _data['describe.1'] + "_" + code + f"_fold{fold}"

        # 判断是否有nan
        if _data.isnull().values.any(): 
            print(i)
            continue

        if 'test_class_f1_0' in list(_data):
            _data[f'test_mean_class_f1'] = (_data[f'test_class_f1_0'] + _data[f'test_class_f1_1']) / 2
        elif 'test_final_class_f1_0' in list(_data):
            # 同时采用 best/final
            d2 = _data.copy()
            d2[f'test_mean_class_f1'] = _data[f'test_best_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                d2[f'test_mean_class_f1'] += _data[f'test_best_class_f1_{i}']
            d2[f'test_mean_class_f1'] = d2[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            d2['train_title'] += '_best'

            _data[f'test_mean_class_f1'] = _data[f'test_final_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                _data[f'test_mean_class_f1'] += _data[f'test_final_class_f1_{i}']
            _data[f'test_mean_class_f1'] = _data[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            _data['train_title'] += '_final'

            _data = pd.concat([_data, d2], ignore_index=True)

        # for _type in ['train', 'val', 'test_best', 'test_final', 'test_dummy']:
        #     _data[f'{_type}_mean_class_f1'] = (_data[f'{_type}_class_f1_0'] + _data[f'{_type}_class_f1_1']) / 2

        # for _type in ['train', 'val', 'test_best', 'test_final']:
            # _data[f'{_type}_mean_class_f1_enhanced_pct'] = 100 * (_data[f'{_type}_mean_class_f1'] - _data['test_dummy_mean_class_f1']) / _data['test_dummy_mean_class_f1']

        data = pd.concat([data, _data], ignore_index=True)

try:
    data['mean_test_01_f1'] = (data['test_best_class_f1_0'] + data['test_best_class_f1_1']) / 2
    data['mean_val_01_f1'] = (data['val_class_f1_0'] + data['val_class_f1_1']) / 2
except:
    pass

# 只使用 top 数据集 predict_n100
# data = data.loc[data['train_title'].str.contains('top5')].reset_index(drop=True)
# data = data.loc[data['train_title'].str.contains('predict_n100')].reset_index(drop=True)

data

,train_title,root,alist_upload_folder,amp,epochs,batch_n,batch_size,learning_rate,abs_learning_rate,no_better_stop,...,test_dummy_mean_class_f1,train_mean_class_f1_enhanced_pct,val_mean_class_f1_enhanced_pct,test_best_mean_class_f1_enhanced_pct,test_final_mean_class_f1_enhanced_pct,each_epoch_cost,cost,test_mean_class_f1,mean_test_01_f1,mean_val_01_f1
0,20250831_2_P100_seed0_wd0.0001_IDX2_final,20250831_2_P100_seed0_wd0.0001_IDX2,train_data,no,120,128,8192,0.0384,0,0,...,0.498818,98.590446,43.115415,51.297751,47.526875,0.03h,3.40h,0.734614,0.754701,0.713886
1,20250831_2_P100_seed0_wd0.0001_IDX2_best,20250831_2_P100_seed0_wd0.0001_IDX2,train_data,no,120,128,8192,0.0384,0,0,...,0.498818,98.590446,43.115415,51.297751,47.526875,0.03h,3.40h,0.759017,0.754701,0.713886
2,20250831_2_P100_seed0_wd0.001_IDX1_final,20250831_2_P100_seed0_wd0.001_IDX1,train_data,no,120,128,8192,0.0384,0,0,...,0.498818,97.891898,41.274842,49.598287,46.969679,0.03h,3.41h,0.727943,0.746224,0.704705
3,20250831_2_P100_seed0_wd0.001_IDX1_best,20250831_2_P100_seed0_wd0.001_IDX1,train_data,no,120,128,8192,0.0384,0,0,...,0.498818,97.891898,41.274842,49.598287,46.969679,0.03h,3.41h,0.733610,0.746224,0.704705
4,20250831_2_P100_seed0_wd0.01_IDX0_final,20250831_2_P100_seed0_wd0.01_IDX0,train_data,no,120,128,8192,0.0384,0,0,...,0.498818,89.222979,38.945215,58.078636,44.823686,0.03h,3.21h,0.724381,0.788525,0.693084
5,20250831_2_P100_seed0_wd0.01_IDX0_best,20250831_2_P100_seed0_wd0.01_IDX0,train_data,no,120,128,8192,0.0384,0,0,...,0.498818,89.222979,38.945215,58.078636,44.823686,0.03h,3.21h,0.775883,0.788525,0.693084
6,20250831_2_P100_seed1_wd0.0001_IDX5_final,20250831_2_P100_seed1_wd0.0001_IDX5,train_data,no,120,128,8192,0.0384,0,0,...,0.499773,98.212574,34.638305,59.020688,42.031013,0.03h,3.43h,0.693072,0.794742,0.672886
7,20250831_2_P100_seed1_wd0.0001_IDX5_best,20250831_2_P100_seed1_wd0.0001_IDX5,train_data,no,120,128,8192,0.0384,0,0,...,0.499773,98.212574,34.638305,59.020688,42.031013,0.03h,3.43h,0.800929,0.794742,0.672886
8,20250831_2_P100_seed1_wd0.001_IDX4_final,20250831_2_P100_seed1_wd0.001_IDX4,train_data,no,120,128,8192,0.0384,0,0,...,0.499773,97.460177,43.649353,58.163714,49.782550,0.03h,3.19h,0.749514,0.790459,0.717920
9,20250831_2_P100_seed1_wd0.001_IDX4_best,20250831_2_P100_seed1_wd0.001_IDX4,train_data,no,120,128,8192,0.0384,0,0,...,0.499773,97.460177,43.649353,58.163714,49.782550,0.03h,3.19h,0.795206,0.790459,0.717920


In [32]:
if len(folders) == 1:
    data.to_csv(os.path.join(folders[0], 'result.csv'), index=False)


In [33]:
list(data)

['train_title',
 'root',
 'alist_upload_folder',
 'amp',
 'epochs',
 'batch_n',
 'batch_size',
 'learning_rate',
 'abs_learning_rate',
 'no_better_stop',
 'checkpointing_steps',
 'label_smoothing',
 'weight_decay',
 'classify',
 'y_n',
 'debug',
 'test',
 'seed',
 'label_train',
 'label_val',
 'label_test_final',
 'model',
 'grad_norm_best',
 'grad_norm',
 'total_grad_norm_best',
 'total_grad_norm',
 'train_loss_best',
 'train_loss',
 'train_acc_best',
 'train_acc',
 'train_f1_best',
 'train_f1',
 'train_recall_best',
 'train_recall',
 'train_class_acc_0_best',
 'train_class_acc_0',
 'train_class_mcc_0_best',
 'train_class_mcc_0',
 'train_class_f1_0_best',
 'train_class_f1_0',
 'train_class_recall_0_best',
 'train_class_recall_0',
 'train_class_acc_1_best',
 'train_class_acc_1',
 'train_class_mcc_1_best',
 'train_class_mcc_1',
 'train_class_f1_1_best',
 'train_class_f1_1',
 'train_class_recall_1_best',
 'train_class_recall_1',
 'val_loss_best',
 'val_loss',
 'val_acc_best',
 'val_acc',

In [34]:
data[['train_title', 'label_train']]

,train_title,label_train
0,20250831_2_P100_seed0_wd0.0001_IDX2_final,1(364471)@1(364471)
1,20250831_2_P100_seed0_wd0.0001_IDX2_best,1(364471)@1(364471)
2,20250831_2_P100_seed0_wd0.001_IDX1_final,1(364471)@1(364471)
3,20250831_2_P100_seed0_wd0.001_IDX1_best,1(364471)@1(364471)
4,20250831_2_P100_seed0_wd0.01_IDX0_final,1(364471)@1(364471)
5,20250831_2_P100_seed0_wd0.01_IDX0_best,1(364471)@1(364471)
6,20250831_2_P100_seed1_wd0.0001_IDX5_final,1(364471)@1(364471)
7,20250831_2_P100_seed1_wd0.0001_IDX5_best,1(364471)@1(364471)
8,20250831_2_P100_seed1_wd0.001_IDX4_final,1(364471)@1(364471)
9,20250831_2_P100_seed1_wd0.001_IDX4_best,1(364471)@1(364471)


In [37]:
# 多seed平均
key_col_name = ['train_f1']
key_col_name = ['train_loss', 'train_f1', 'val_f1', 'val_f1_best', 'val_loss', 'test_best_f1', 'label_train']
# key_col_name = ['train_loss', 'train_f1', 'val_f1', 'val_f1_best', 'val_loss', 'label_train']

d = data.loc[:, key_col_name + ['cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'seed' not in i]))
d['label_train'] = d['label_train'].apply(lambda x:int(x.split('@')[-1][2:-1])*2)

# 只考察 final
_match_bool = data['train_title'].apply(lambda x: 'final' in x or ('final' not in x and 'best' not in x))
d = d.loc[_match_bool]
# 去除 final/best 的标记
d['train_title'] = d['train_title'].apply(lambda x: x.replace('_final', '').replace('_best', ''))

# # 关键字过滤 白名单
# keys = ['bc']
# _match_key_bool = data['train_title'].apply(lambda x: any([k in x for k in keys]))
# d = d.loc[_match_key_bool]
# # 关键字过滤 黑名单
# keys = ['top', '200', '300']
# _match_key_bool = data['train_title'].apply(lambda x: all([k not in x for k in keys]))
# d = d.loc[_match_key_bool]

# # 根据 idx 区分类别
# # 用于未区分title的情况
# _cls = d['train_title'].apply(lambda x: [(int(i.replace('IDX', '')) % 3) for i in x.split('_') if 'IDX' in i][0])
# _cls = _cls.apply(lambda x: ['nomove30', 'nomove50', 'nomove70'][x])
# d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i])) + '_' + _cls

# title 已经区分
d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(round(x,2)) + 'h')
d.sort_values('val_f1_best',ascending=False)


,train_loss,train_f1,val_f1,val_f1_best,val_loss,test_best_f1,label_train,cost
train_title,,,,,,,,
20250831_2_P100_wd0.01,0.140439,0.943630,0.699601,0.755078,1.147292,0.789186,728942.0,3.31h
20250831_2_P100_wd0.0001,0.026638,0.990275,0.689474,0.730915,2.591012,0.767236,728942.0,3.37h
20250831_2_P100_wd0.001,0.035946,0.986799,0.713429,0.726419,2.085281,0.763554,728942.0,3.25h


In [36]:
# 多IDX平均
key_col_name = 'val_f1'
key_col_name = 'label_train'

d = data.loc[:, [key_col_name, 'cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i and 'seed' not in i ]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values([key_col_name],ascending=False)

TypeError: agg function failed [how->mean,dtype->object]